In [1]:
%load_ext autoreload

In [2]:
import sys
import os
import json
import pandas as pd
import numpy as np
from collections import defaultdict

os.environ['CUDA_VISIBLE_DEVICES']='0'
pd.set_option('display.max_colwidth', None)
sys.path.append('/home/sile2804/acl21-counter-argument-generation-by-attacking-weak-premises/code/lib/counter_gen_lib/')
sys.path.append('/home/sile2804/acl21-counter-argument-generation-by-attacking-weak-premises/code/lib')

In [3]:
%autoreload
import interact
import utility
import json
from argparse import Namespace
from tabulate import tabulate

In [4]:
ceph_dir = '/home/sile2804/data-ceph/arguana/arg-generation/multi-taks-counter-argument-generation'
local_home_dir = '../data'

In [5]:
data_unique_path = '/reddit_data/conclusion_and_ca_generation/test_conclusion_all_preprocessed.pkl'
#data_unique_path = '/reddit_data/conclusion_and_ca_generation/sample_test_conclusion_all_preprocessed.pkl'
data_path = '/reddit_data/conclusion_and_ca_generation/test_conclusion_all.pkl'

In [6]:
test_df = pd.read_pickle(ceph_dir + data_unique_path)

In [7]:
def convert_jo_to_json(df):
    ranking_problems = []
    for idx, row in df.iterrows():
            documents = [{"relevance":-1, "docText": sent} for sent in row['post']]
            
            ranking_problems.append({
                "queryText": row['title'],
                "documents": documents
            })
    
    return {
        "rankingProblems": ranking_problems
    }

In [21]:
json_data = convert_jo_to_json(test_df)
json.dump(json_data, open('../data/output/arg-undermining-baseline/json_valid.json', 'w'))

### Predict premise attackabaility:

In [ ]:
! python /home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/thirdparty/ranking/tensorflow_ranking/extension/examples/tfrbert_client_predict_from_json.py \
    --vocab_file=/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/thirdparty/uncased_L-12_H-768_A-12_TF2/vocab.txt \
    --sequence_length=128 \
    --input_file=../data/output/arg-undermining-baseline/json_valid.json \
    --output_file=../data/output/arg-undermining-baseline/json_valid_with_attacked_premises.json \
    --do_lower_case

 * Running with arguments: Namespace(do_lower_case=True, input_file='../data/output/arg-undermining-baseline/json_valid.json', output_file='../data/output/arg-undermining-baseline/json_valid_with_attacked_premises.json', sequence_length=128, server_port='8503', vocab_file='/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/thirdparty/uncased_L-12_H-768_A-12_TF2/vocab.txt')
 * Generating predictions for JSON ranking problems (filename: ../data/output/arg-undermining-baseline/json_valid.json)

Predicting 1 / 8533 (0.01%)
Predicting 2 / 8533 (0.02%)
Predicting 3 / 8533 (0.04%)
Predicting 4 / 8533 (0.05%)
Unexpected error: <class 'grpc._channel._InactiveRpcError'>
Predicting 5 / 8533 (0.06%)
Predicting 6 / 8533 (0.07%)
Predicting 7 / 8533 (0.08%)
Predicting 8 / 8533 (0.09%)
Predicting 9 / 8533 (0.11%)
Unexpected error: <class 'grpc._channel._InactiveRpcError'>
Predicting 10 / 8533 (0.12%)
Predicting 11 / 8533 (0.13%)
Unexpected error: <class 'grpc._channel._Inactiv

### Generate counters

In [11]:
#load weak premises
weak_premises = json.load(open('../data/output/arg-undermining-baseline/json_valid_with_attacked_premises.json'))

In [12]:
title_to_comments = {x['queryText']: [d for d in x['documents']] for x in weak_premises['rankingProblemsOutput']}

In [13]:
test_df['1_weak_premise'] = test_df.title.apply(lambda x: [x['docText'] for x in sorted(title_to_comments[x], key=lambda x: -x['score'])[:1]])
test_df['3_weak_premises'] = test_df.title.apply(lambda x: [x['docText'] for x in sorted(title_to_comments[x], key=lambda x: -x['score'])[:3]])
test_df['claim'] = test_df.title

In [14]:
test_df[["title", "1_weak_premise"]].head()

,title,1_weak_premise
410850,people should come with instructions,"[i would for example put things like introvert but likes to have fun and sing karaoke and write on reddit dislikes sports, random acts of kindness and illogical thinking]"
410858,People should not be heavily criticized for things they put on social media in the distant past,[i think it's unreasonable to expect anyone to do that.]
410902,We shouldn't focus on slowing climate change,[first of all: climate change is real happening and in large part caused by human activity.]
410910,The Australian PM was right to tell students to stop activism around global warming,"[first things first, just so i don't get mauled alive, i believe in climate change.]"
410916,Feeding cats or dogs a diet with meat is indefensible.,"[even if a vegan diet killed a cat dog immediately which it doesn't , it'd still be the more moral thing to do because you'd kill only 1 animal instead of 100s.]"


In [15]:
args = Namespace(max_length=100, device='cuda:0', temperature=1.0, 
                 top_k=50, top_p=0.95, no_sample=False, min_length=50, premise_extra=False,
)

baseline_path = '/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/gen_models/Dec10_21-48-03_gammaweb03_jo_baseline_gpt/'
app_v2_path = '/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/gen_models/Dec10_21-50-50_gammaweb03_my_approach_v2/'
app_v3_path = '/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/gen_models/Dec10_21-53-13_gammaweb03_my_approach_v3/'
app_v4_path = '/home/sile2804/data-ceph/arguana/arg-generation/argument-undermining/models/gen_models/Dec21_17-10-34_gammaweb03_my_approach_v4/'

In [16]:
#test_df = test_df.sample(10)

In [ ]:
args.build_instance_version = 'v2'
test_df = utility.perform_attacks_hua_df(test_df, app_v2_path, args, 'app_v2_attack_on_weak_premise', 'title+full_post', '1_weak_premise', baseline=False)
test_df = utility.perform_attacks_hua_df(test_df, app_v2_path, args, 'app_v2_attack_on_3_weak_premises', 'title+full_post', '3_weak_premises', baseline=False)

/home/sile2804/acl21-counter-argument-generation-by-attacking-weak-premises/code/lib/counter_gen_lib/interact.py:109: UserWarning: Warning: model generating special token with probability 1.
  warnings.warn("Warning: model generating special token with probability 1.")
/home/sile2804/acl21-counter-argument-generation-by-attacking-weak-premises/code/lib/counter_gen_lib/interact.py:79: UserWarning: Warning: Weak premise couldn't be identified in argument. Attacking the first sentence
  warnings.warn("Warning: Weak premise couldn't be identified in argument. Attacking the first sentence")
/home/sile2804/acl21-counter-argument-generation-by-attacking-weak-premises/code/lib/counter_gen_lib/interact.py:95: UserWarning: Warning: sequense is larger than 512
  warnings.warn("Warning: sequense is larger than 512")


In [ ]:
test_df.to_pickle('../data/output/arg-undermining-baseline/test_preds.pkl')

------

In [9]:
test_df = pd.read_pickle('../data/output/arg-undermining-baseline/test_preds.pkl')

In [ ]:
test_df[['title', 'app_v3_attack_on_3_weak_premises']].head()